# Test Case LVV-t1992
This case will verify that the M2/Camera Hexapod is minimally functional.
The blocks below represent different steps of the test case.

Requirements
* EFD
* Hexapod(s) powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import Hexapod
import logging
import asyncio

In [2]:
STD_WAIT = 39

In [3]:
log = logging.getLogger(__name__)

In [4]:
domain = salobj.Domain()
hexapod_csc = salobj.Remote(name="Hexapod", domain=domain, index=1)
await hexapod_csc.start_task

In [5]:
def enabled_substate_callback(evt):
    print(Hexapod.EnabledSubstate(evt.enabledSubstate))

In [ ]:
# inPosition is not published due to issue in XML 4.4 which is fixed in later revisions of the XML
def in_position_callback(evt):
    print(evt.in_position)

In [6]:
connected = await hexapod_csc.evt_connected.aget()
commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.")

controller_state = await hexapod_csc.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate

if state == salobj.State.OFFLINE and offline_substate == Hexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate")

commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)

RemoteTelemetry(Hexapod, 1, Electrical) falling behind; read 100 messages


[<State.ENABLED: 2>]

RemoteTelemetry(Hexapod, 1, Application) falling behind; read 100 messages


# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [7]:
hexapod_csc.evt_controllerState.callback = enabled_substate_callback
await hexapod_csc.cmd_positionSet.set_start(x=0,y=0,z=200, u=0,v=0,w=0,sync=True)
await hexapod_csc.cmd_positionSet.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
await hexapod_csc.cmd_move.set_start()



RemoteTelemetry(Hexapod, 1, Actuators) falling behind; read 100 messages
RemoteEvent(Hexapod, 1, heartbeat) falling behind; read 100 messages


In [8]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

RemoteTelemetry(Hexapod, 1, Actuators) falling behind; read 100 messages


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# thermal sensors

In [13]:
await hexapod_csc.cmd_positionSet.set_start(x=0,y=0,z=5000,u=0,v=0,w=0,sync=True)
await hexapod_csc.cmd_move.set_start()
await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()
# cs = await hexapod_csc.evt_controllerState.aget()
# print(Hexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != Hexapod.EnabledSubstate.CONTROLLED_STOPPING:
#     raise Exception(f"Controller's substate is not {Hexapod.EnabledSubstate.CONTROLLED_STOPPING}. It is {Hexapod.EnabledSubstate(cs.enabledSubstate)}")
# cs = await hexapod_csc.evt_controllerSubstate.aget()
# print(Hexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != Hexapod.EnabledSubstate.STATIONARY:
#     raise Exception(f"Controller's substate is not {Hexapod.EnabledSubstate.STATIONARY}. It is {Hexapod.EnabledSubstate(cs.enabledSubstate)}")


RemoteTelemetry(Hexapod, 1, Actuators) falling behind; read 100 messages
RemoteTelemetry(Hexapod, 1, Electrical) falling behind; read 100 messages
RemoteTelemetry(Hexapod, 1, Application) falling behind; read 100 messages
RemoteEvent(Hexapod, 1, heartbeat) falling behind; read 100 messages


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


In [10]:
await asyncio.sleep(STD_WAIT)

RemoteTelemetry(Hexapod, 1, Actuators) falling behind; read 100 messages
RemoteTelemetry(Hexapod, 1, Electrical) falling behind; read 100 messages
RemoteTelemetry(Hexapod, 1, Application) falling behind; read 100 messages
RemoteEvent(Hexapod, 1, heartbeat) falling behind; read 17 messages


# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [24]:
await hexapod_csc.cmd_positionSet.set_start(x=0,y=0,z=200,u=0,v=0,w=0)
await hexapod_csc.cmd_positionSet.set_start(x=0,y=0,z=800,u=0,v=0,w=0)
await hexapod_csc.cmd_moveLUT.set_start(az=180,elev=60, temp=10)

RemoteTelemetry(Hexapod, 1, Electrical) falling behind; read 100 messages
RemoteTelemetry(Hexapod, 1, Application) falling behind; read 100 messages
RemoteTelemetry(Hexapod, 1, Actuators) falling behind; read 100 messages
RemoteEvent(Hexapod, 1, heartbeat) falling behind; read 100 messages


In [ ]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [11]:
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0)
await hexapod_csc.cmd_move.set_start()
# cs = await hexapod_csc.evt_controllerState.aget()
# if cs.enabledSubstate != hexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT:
#     raise Exception(f"Controller's substate is not {hexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT}. It is {hexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# cs = await hexapod_csc.evt_controllerSubstate.aget()
# if cs.enabledSubstate != hexapod.SetEnabledSubstateParam.STATIONARY:
#     raise Exception(f"Controller's substate is not {hexapod.SetEnabledSubstateParam.STATIONARY}. It is {hexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# in_position = await hexapod_csc.evt_inPosition.aget(timeout=10)
# if in_position.inPosition is False:
#     raise Exception("Controller not in position.")

RemoteTelemetry(Hexapod, 1, Actuators) falling behind; read 100 messages
RemoteTelemetry(Hexapod, 1, Electrical) falling behind; read 100 messages
RemoteTelemetry(Hexapod, 1, Application) falling behind; read 100 messages
RemoteEvent(Hexapod, 1, heartbeat) falling behind; read 100 messages


In [12]:
await asyncio.sleep(STD_WAIT)

RemoteTelemetry(Hexapod, 1, Application) falling behind; read 100 messages


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


RemoteEvent(Hexapod, 1, heartbeat) falling behind; read 35 messages


# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |